In [1]:
import os, pandas as pd, numpy as np
from datetime import datetime as dt

In [2]:
datapath = '../data/raw/jawiki/dumps_unzipped/'

In [3]:
mtdt = pd.read_csv('../references/metadata/history.tsv', sep='\t')
# print(mtdt.col_name.tolist())

### head

In [4]:
with open(datapath+'2021-12.jawiki.2013.tsv', 'r') as file:
    for i in range(3):
        print(file.readline())

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/jawiki/dumps_unzipped/2021-12.jawiki.2013.tsv'

### split file so I can run csvsql to generate schema

#### copy 500 rows of the tsv

In [7]:
ct = 0
with open(datapath+'2021-12.jawiki.2013.tsv', 'r') as infile:
    with open('/home/bhrdwj/git/predwikt/data/processed/scratch_for_schema.tsv', 'w+') as outfile:
        while ct < 3: #5*10**2:
            outfile.write(infile.readline())
            ct += 1
print(ct)

3


#### confirm the rows were copied

In [8]:
ct = 0
with open('/home/bhrdwj/git/predwikt/data/processed/scratch_for_schema.tsv', 'r') as file:
    for line in file:
        ct += 1
print(ct)

3


#### hand-edited query

In [35]:
make_history_table = """
CREATE TABLE ja_hist ( 
wiki_db VARCHAR(20) NOT NULL,
event_entity VARCHAR(20) NOT NULL,
event_type VARCHAR(20) NOT NULL,
event_timestamp VARCHAR(30) NOT NULL,
event_comment TEXT,
event_user_id BIGINT,
event_user_text_historical VARCHAR(80),
event_user_text VARCHAR(80),
event_user_blocks_historical VARCHAR(30),
event_user_blocks VARCHAR(60),
event_user_groups_historical VARCHAR(60),
event_user_groups VARCHAR(200),
event_user_is_bot_by_historical VARCHAR(10),
event_user_is_bot_by VARCHAR(10),
event_user_is_created_by_self BOOL,
event_user_is_created_by_system BOOL,
event_user_is_created_by_peer BOOL,
event_user_is_anonymous BOOL,
event_user_registration_timestamp VARCHAR(30),
event_user_creation_timestamp VARCHAR(30),
event_user_first_edit_timestamp VARCHAR(30),
event_user_revision_count BIGINT,
event_user_seconds_since_previous_revision BIGINT,
page_id BIGINT,
page_title_historical VARCHAR(200),
page_title VARCHAR(200),
page_namespace_historical INT,
page_namespace_is_content_historical BOOL,
page_namespace INT,
page_namespace_is_content BOOL,
page_is_redirect BOOL,
page_is_deleted BOOL,
page_creation_timestamp VARCHAR(30),
page_first_edit_timestamp VARCHAR(30),
page_revision_count BIGINT,
page_seconds_since_previous_revision BIGINT,
user_id BIGINT,
user_text_historical VARCHAR(100),
user_text VARCHAR(100),
user_blocks_historical VARCHAR(50),
user_blocks VARCHAR(50),
user_groups_historical VARCHAR(50),
user_groups VARCHAR(50),
user_is_bot_by_historical VARCHAR(50),
user_is_bot_by VARCHAR(50),
user_is_created_by_self BOOL,
user_is_created_by_system BOOL,
user_is_created_by_peer BOOL,
user_is_anonymous BOOL,
user_registration_timestamp VARCHAR(30),
user_creation_timestamp VARCHAR(30),
user_first_edit_timestamp VARCHAR(30),
revision_id BIGINT,
revision_parent_id BIGINT,
revision_minor_edit BOOL,
revision_deleted_parts VARCHAR(30),
revision_deleted_parts_are_suppressed BOOL,
revision_text_bytes BIGINT,
revision_text_bytes_diff BIGINT,
revision_text_sha1 VARCHAR(50),
revision_content_model BOOL,
revision_content_format BOOL,
revision_is_deleted_by_page_deletion BOOL,
revision_deleted_by_page_deletion_timestamp VARCHAR(30),
revision_is_identity_reverted BOOL,
revision_first_identity_reverting_revision_id BIGINT,
revision_seconds_to_identity_revert BIGINT,
revision_is_identity_revert BOOL,
revision_is_from_before_page_creation BOOL,
revision_tags VARCHAR(50),
row_id BIGINT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (row_id)
);
"""

#### simpler table

### create table ja_hist

#### imports, login

In [36]:
import mysql.connector as mysql
import sqlalchemy

```bash
mysql --user=root --password=XXXXXXXX
```

```sql
CREATE DATABASE ja_hist CHARACTER SET utf8 COLLATE utf8_bin;
USE ja_wiki;
GRANT ALL PRIVILEGES ON ja_hist TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
GRANT ALL PRIVILEGES ON ja_hist.* TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
EXIT
```

In [14]:
mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')

#### connect to database

In [37]:
host='localhost'; user=mysql_superuser; passwd=mysql_su_pass; dbname='ja_hist';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine1 = sqlalchemy.create_engine(connection_str)
    conn1 = engine1.connect()
except:
    print('Database connection error - check creds')
metadata = sqlalchemy.MetaData(conn1)
metadata.reflect()
metadata.tables.keys()

dict_keys(['ja_hist'])

#### create table with schema

In [38]:
conn1.execute('DROP TABLE IF EXISTS ja_hist;')
conn1.execute(make_history_table)

In [41]:
a = conn1.execute('describe ja_hist;')
a.all();

### insert data

Reference: ([link](https://mariadb.com/kb/en/load-data-infile/))  
```sql
LOAD DATA INFILE 'ld.txt' INTO TABLE ld (@i,v) SET i=@i*2;
```

In [44]:
load_data_sql = """
LOAD DATA LOCAL INFILE '/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.2013.tsv'
INTO TABLE ja_hist 
CHARACTER SET utf8mb4
FIELDS TERMINATED BY "\t";
"""

In [45]:
conn1.execute(load_data_sql)

START 19:24PM

In [46]:
tblname='ja_hist'
sql_eda2 = f'select count(*) from {tblname};'
print(dt.now())
sql_rows = pd.read_sql(sql_eda2,engine1)
print(sql_rows)
print(dt.now())

2022-01-09 16:01:32.096460
   count(*)
0         0
2022-01-09 16:01:35.371606


In [30]:
a = conn1.execute('select * from ja_hist;')
a.fetchall()

[]

In [ ]:
conn1.execute('insert ')